### **Build Graph Domain**

In [2]:
from tqdm import tqdm

In [48]:
import numpy as np
from queue import PriorityQueue

class GraphDomain:
    def __init__(self, path, name = 'NULL', pad = 1):
        self.NAME = name
        self.NUM_NODE = 0
        self.NUM_DOMAIN = 0
        self.START_NODE = -1
        self.END_NODE = -1

        self.adj = {}
        self.distance = {}
        self.pre_node = {}
        self.domain_start_nodes = {}

        self.pad = pad
        self.load_data(path)
        self.build_graph_Floyd_Warshall()
 
        
    def load_data(self, path):
        f = open(path, 'r')

        l1 = f.readline()
        l1 = l1.strip()
        l1 = l1.split(' ')
        self.NUM_NODE = int(l1[0])
        self.NUM_DOMAIN = int(l1[1])

        l2 = f.readline()
        l2 = l2.strip()
        l2 = l2.split(' ')
        self.START_NODE = int(l2[0])
        self.END_NODE = int(l2[1])

        # fomat edge: start, end, weight, domain
        edges = f.readlines()

        f.close()

        #setup
        for d in range(self.pad, self.pad+self.NUM_DOMAIN):
            self.adj[d] = {}
            self.distance[d] = {}
            self.domain_start_nodes[d] = []
            self.pre_node[d] = {}
            for v in range(self.pad, self.pad+self.NUM_NODE):
                self.adj[d][v] = list()
                self.distance[d][v] = np.full((self.NUM_NODE+self.pad,), np.inf)
                self.distance[d][v][v] = 0
                self.pre_node[d][v] = np.full((self.NUM_NODE+self.pad,), -1)
        
        for e in edges:
            e = e.strip()
            e = e.split(' ')
            #u = e[0], v = e[1], w = e[2], d = e[3]
            self.adj[int(e[3])][int(e[0])].append((int(e[1]), int(e[2])))
            self.distance[int(e[3])][int(e[0])][int(e[1])] = min(self.distance[int(e[3])][int(e[0])][int(e[1])], int(e[2]))
            
    def Floyd_Warshall(self, domain):
        dis = self.distance[domain]
        for k in range(self.pad, self.pad+self.NUM_NODE):
            if len(self.adj[d][k]) != 0:
                self.domain_start_nodes[d].append(k)
            for i in range(self.pad, self.pad+self.NUM_NODE):
                for j in range(self.pad, self.pad+self.NUM_NODE):
                    dis[i][j] = min(dis[i][j], dis[i][k] + dis[k][j])
    
    def build_graph_Floyd_Warshall(self):
        for d in tqdm(range(self.pad, self.NUM_DOMAIN+self.pad)):
            self.Floyd_Warshall(d)
                

    def Dijkstra(self, domain, start, to_show=False):
        if(to_show):
            dis = np.full((self.NUM_NODE+self.pad,), np.inf)
        else:
            dis = self.distance[domain][start]
        # pre = np.full((self.NUM_NODE+self.pad,), -1)
        pre = self.pre_node[domain][start]
        dis[start] = 0
        pre[start] = start

        PQ = PriorityQueue()
        PQ.put((dis[start], start))

        while not PQ.empty():
            curr = PQ.get()
            u = curr[1]
            d = curr[0]

            for e in self.adj[domain][u]:
                if e[1] + d < dis[e[0]]:
                    dis[e[0]] = e[1] + d
                    pre[e[0]] = u
                    PQ.put((dis[e[0]], e[0]))
        
        # return pre
    

    def build_graph(self):
        for d in tqdm(range(self.pad, self.NUM_DOMAIN+self.pad)):
            for v in range(self.pad, self.NUM_NODE+self.pad):
                self.Dijkstra(d, v)
                if len(self.adj[d][v]) != 0:
                    self.domain_start_nodes[d].append(v)
    

In [4]:
test = GraphDomain('D:\MachineLearning\EvolutionComputation\IDPCDU\Datasets\IDPCDU_Edges\set2\idpc_100x200x2296097.idpc')

100%|██████████| 200/200 [04:16<00:00,  1.28s/it]


In [14]:
for d in range(1, 201):
    s = 0
    for i in range(1, 101):
        s += len(test.adj[d][i])
    print(d, s)

1 11516
2 11558
3 11579
4 11655
5 11398
6 11405
7 11593
8 11531
9 11366
10 11362
11 11297
12 11494
13 11554
14 11572
15 11514
16 11518
17 11666
18 11348
19 11552
20 11522
21 11273
22 11445
23 11451
24 11322
25 11621
26 11701
27 11587
28 11475
29 11359
30 11441
31 11463
32 11671
33 11517
34 11436
35 11470
36 11495
37 11545
38 11480
39 11674
40 11351
41 11631
42 11543
43 11373
44 11390
45 11693
46 11724
47 11407
48 11561
49 11548
50 11379
51 11394
52 11482
53 11549
54 11426
55 11402
56 11612
57 11304
58 11307
59 11461
60 11570
61 11632
62 11529
63 11408
64 11564
65 11661
66 11518
67 11535
68 11593
69 11293
70 11545
71 11602
72 11567
73 11498
74 11346
75 11717
76 11291
77 11487
78 11584
79 11469
80 11565
81 11218
82 11392
83 11440
84 11394
85 11624
86 11272
87 11470
88 11327
89 11490
90 11348
91 11464
92 11461
93 11474
94 11509
95 11525
96 11550
97 11200
98 11517
99 11653
100 11523
101 11380
102 11505
103 11470
104 11512
105 11347
106 11483
107 11455
108 11491
109 11411
110 11355
111 1161

In [3]:
len(test.adj)

3

In [4]:
test.pre_node

{1: {1: array([-1,  1,  1,  2, -1,  1, -1,  2, -1, -1]),
  2: array([-1, -1,  2,  2, -1, -1, -1,  2, -1, -1]),
  3: array([-1, -1, -1,  3, -1, -1, -1, -1, -1, -1]),
  4: array([-1, -1, -1, -1,  4, -1, -1, -1, -1, -1]),
  5: array([-1, -1, -1, -1, -1,  5, -1, -1, -1, -1]),
  6: array([-1, -1, -1, -1, -1, -1,  6, -1,  6, -1]),
  7: array([-1, -1, -1, -1, -1, -1, -1,  7, -1, -1]),
  8: array([-1, -1, -1, -1, -1, -1,  8, -1,  8, -1]),
  9: array([-1, -1, -1, -1, -1, -1, -1, -1, -1,  9])},
 2: {1: array([-1,  1,  1, -1,  5,  7,  7,  2, -1,  2]),
  2: array([-1,  5,  2, -1,  5,  7,  7,  2, -1,  2]),
  3: array([-1, -1, -1,  3, -1, -1, -1, -1, -1, -1]),
  4: array([-1, -1, -1, -1,  4, -1, -1, -1, -1, -1]),
  5: array([-1,  5,  1, -1,  5,  5,  5,  2, -1,  2]),
  6: array([-1, -1, -1, -1, -1, -1,  6, -1, -1, -1]),
  7: array([-1,  5,  1, -1,  5,  7,  7,  7, -1,  2]),
  8: array([-1, -1, -1, -1, -1, -1, -1, -1,  8,  8]),
  9: array([-1, -1, -1, -1, -1, -1, -1, -1, -1,  9])},
 3: {1: array([-1,  

In [5]:
test.distance

{1: {1: array([inf,  0.,  1.,  3., inf,  5., inf,  3., inf, inf]),
  2: array([inf, inf,  0.,  2., inf, inf, inf,  2., inf, inf]),
  3: array([inf, inf, inf,  0., inf, inf, inf, inf, inf, inf]),
  4: array([inf, inf, inf, inf,  0., inf, inf, inf, inf, inf]),
  5: array([inf, inf, inf, inf, inf,  0., inf, inf, inf, inf]),
  6: array([inf, inf, inf, inf, inf, inf,  0., inf,  1., inf]),
  7: array([inf, inf, inf, inf, inf, inf, inf,  0., inf, inf]),
  8: array([inf, inf, inf, inf, inf, inf,  2., inf,  0., inf]),
  9: array([inf, inf, inf, inf, inf, inf, inf, inf, inf,  0.])},
 2: {1: array([inf,  0.,  3., inf,  8.,  6.,  5.,  4., inf,  5.]),
  2: array([inf,  5.,  0., inf,  5.,  3.,  2.,  1., inf,  2.]),
  3: array([inf, inf, inf,  0., inf, inf, inf, inf, inf, inf]),
  4: array([inf, inf, inf, inf,  0., inf, inf, inf, inf, inf]),
  5: array([inf,  2.,  5., inf,  2.,  0.,  2.,  6., inf,  7.]),
  6: array([inf, inf, inf, inf, inf, inf,  0., inf, inf, inf]),
  7: array([inf,  4.,  7., inf,  

In [6]:
test.Dijkstra(1, 1, to_show=True)

### **Encoding Version 1**

####     Gene has  two parts:
*   Priority of the Domain
*   End node of the domain

In [ ]:
import numpy as np


class AlgorithmV1(GraphDomain):


##################################### Version 1 ########################################
#                                 Encoding Version 1                                   #
# Gene has  two parts:
#    Priority of the Domain
#    End node of the domain

    def Decode(self, indiv):
        if len(indiv[0]) == self.NUM_DOMAIN: return indiv

        result = [[], []]
        for i in range(len(indiv[0])):
            if indiv[0][i] <= self.NUM_DOMAIN:
                result[0].append(indiv[0][i])
                result[1].append(indiv[1][i])
        return np.array(result)

    def Cost(self, indiv):
        indiv = self.Decode(indiv)

        path = []
        weight = np.inf
        domains = np.argsort(-np.array(indiv[0])) + 1
        curr_domain = domains[0]
        end_domains = []   #(domain, weight, index in path)
        start_node = self.START_NODE

        for next_domain in domains[1:]:
            start_node_set = []
            for v in self.domain_start_nodes[next_domain]:
                if self.distance[curr_domain][start_node][v] != np.inf:
                    start_node_set.append(v)

            dis_to_end = self.distance[curr_domain][start_node][self.END_NODE]
            if  dis_to_end != np.inf:
                end_domains.append(curr_domain, weight+dis_to_end, len(path))

            if len(start_node_set) == 0:
                if len(end_domains) == 0:
                    continue
                else:
                    break

            end_node = start_node_set[int(indiv[1][curr_domain-1])%len(start_node_set)]
            weight += self.distance[curr_domain][start_node][end_node]
            path.append((curr_domain, start_node, end_node))
            curr_domain = next_domain
            start_node = end_node

        dis_to_end = self.distance[curr_domain][start_node][self.END_NODE]
        if  dis_to_end != np.inf :
            end_domains.append(curr_domain, weight+dis_to_end, len(path))
        
        if len(end_domains) == 0:
            return path, np.inf
        else:
            endDomain = min(end_domains, key=lambda tup: tup[1])
            path[endDomain[2]] = (endDomain[0], path[endDomain[2]][1], self.END_NODE)
            
            return path[:endDomain[2]+1], endDomain[1]


    def show_path(self, indiv):
        path, weight = self.Cost(indiv)

        res = {}
        for pi in path:
            path_i = []
            domain = pi[0]
            start = pi[1]
            end = pi[2]
            pre = self.pre_node[domain][start]
            curr = end
            while(pre[curr] != curr):
                path_i.append(curr)
                curr = pre[curr]
            path_i.append(start)
            res[domain] = path_i[::-1]

        if(weight != np.inf):
            print(res)
        else:
            print('Error path: Not find path to end node!')
        return res, weight

        

### **Encoding Version 2 - Encode Priority**

In [69]:
import numpy as np
from queue import PriorityQueue


class EncodePriority(GraphDomain):

##################################### Version 2 ########################################
#                                 Encode Priority                                      #


    def Decode(self, indiv):
        if len(indiv) == self.NUM_DOMAIN: return indiv
        result = []
        for i in indiv:
            if i <= self.NUM_DOMAIN:
                result.append(i)
        return np.array(result)


    def Cost(self, indiv):
        domains = self.Decode(indiv)

        dis = np.full((self.NUM_NODE+1,), np.inf)
        dis[self.START_NODE] = 0

        for d in domains:
            PQ = PriorityQueue()
            for u in self.domain_start_nodes[d]:
                if (dis[u] != np.inf):
                    PQ.put((dis[u], u))

            while not PQ.empty():
                top = PQ.get()
                u = top[1]  
                for i in range(1, 1+self.NUM_NODE):
                    if dis[u] + self.distance[d][u][i] < dis[i]:
                        dis[i] = dis[u] + self.distance[d][u][i]
        
        return dis[self.END_NODE]


    def Greedy_Cost(self, indiv, th = 5):
        domains = self.Decode(indiv)

        dis = np.full((self.NUM_NODE+1,), np.inf)
        dis[self.START_NODE] = 0

        for d in domains:
            PQ = PriorityQueue()
            for u in self.domain_start_nodes[d]:
                if (dis[u] != np.inf):
                    PQ.put((dis[u], u))
            t = 0
            while not PQ.empty() and t < th:
                top = PQ.get()
                u = top[1]  
                for i in range(1, 1+self.NUM_NODE):
                    if dis[u] + self.distance[d][u][i] < dis[i]:
                        dis[i] = dis[u] + self.distance[d][u][i]
                t += 1

        return dis[self.END_NODE]

    def Greedy_CostV2(self, indiv, th = 5):
        domains = self.Decode(indiv)

        dis = np.full((self.NUM_NODE+1,), np.inf)
        dis[self.START_NODE] = 0
        curr_domain = domains[0]
        start_PQ = PriorityQueue()
        start_PQ.put((dis[self.START_NODE], self.START_NODE))

        for next_domain in domains[1:]:
            end_PQ = PriorityQueue()

            t = 0
            while not start_PQ.empty() and t < th:
                top = start_PQ.get()
                u = top[1]

                for i in range(1, 1+self.NUM_NODE):
                    if dis[u] + self.distance[curr_domain][u][i] < dis[i]:
                        dis[i] = dis[u] + self.distance[curr_domain][u][i]
                for i in self.domain_start_nodes[next_domain]:
                    if self.distance[curr_domain][u][i] != np.inf:
                        end_PQ.put((dis[i], i))
                t += 1

            start_PQ = end_PQ
            curr_domain = next_domain

        while not start_PQ.empty():
            top = start_PQ.get()
            u = top[1]

            for i in range(1, 1+self.NUM_NODE):
                if dis[u] + self.distance[curr_domain][u][i] < dis[i]:
                    dis[i] = dis[u] + self.distance[curr_domain][u][i]        
    
        return dis[self.END_NODE]

    def show_path(self, indiv):
        # find path                  
        indiv = self.Decode(indiv)
        domains = np.argsort(-np.array(indiv)) + 1

        dis = np.full((self.NUM_NODE+1,), np.inf)
        end_domain = np.full((self.NUM_NODE+1,), -1)            #end domain of shortest path to the node
        pre = np.full((self.NUM_DOMAIN+1, self.NUM_NODE+1, 2), -1)
        dis[self.START_NODE] = 0
        end_domain[self.START_NODE] = 0

        for d in domains:
            for u in self.domain_start_nodes[d]:
                if (dis[u] != np.inf):             
                    for i in range(1, 1+self.NUM_NODE):
                        if dis[u] + self.distance[d][u][i] < dis[i]:
                            dis[i] = dis[u] + self.distance[d][u][i]
                            end_domain[i] = d
                            pre[d][i][0] = end_domain[u]
                            pre[d][i][1] = u

        # show path
        res = []
        curr = (end_domain[self.END_NODE], self.END_NODE)

        while curr[1] != self.START_NODE:
            domain = curr[0]
            start_node = pre[curr[0]][curr[1]][1]
            path = []

            pre_node_in_domain = self.pre_node[domain][start_node]
            curr_node = curr[1]
            while(pre_node_in_domain[curr_node] != curr_node):
                path.append(curr_node)
                curr_node = pre_node_in_domain[curr_node]
            path.append(start_node)

            res.append({domain: path[::-1]})

            curr = pre[curr[0]][curr[1]]
            
        if(dis[self.END_NODE] != np.inf):
            print(res)
        else:
            print('Error path: Not find path to end node!')
        return res, dis[self.END_NODE]
        

In [20]:
Task = EncodePriority('test.txt')

100%|██████████| 3/3 [00:00<00:00, 3011.71it/s]


In [74]:
def GeneratorPopulaion(sizePop, dims=Task.NUM_DOMAIN):
    Populaion = []
    for i in range(sizePop):
        pi = np.random.permutation(range(1, dims+1))
        f_pi = Task.Greedy_Cost(pi, th=2)
        Populaion.append((pi, f_pi))

    return Populaion

In [73]:
Task.NUM_DOMAIN

200

In [75]:
GeneratorPopulaion(100)

[(array([ 43,  83, 145,  61, 125, 106,  29,  32, 135,  58,  98,  42, 128,
          68,  27,  96, 200, 137, 161,  15,  56,  92, 189,  45,  37, 129,
         143, 172,  82,  89, 140, 124, 152,  73, 103,  28, 115, 136, 141,
         150, 182,  13,  99, 138,  18,  64,  66, 112,  77,  62, 188,  33,
          78,  16,  17,  91,  85, 105,  39,  59,  41,  53, 119, 170,  52,
          63, 122, 144,  46, 199,  54,  87,  19,  22,  49, 110,  30,   8,
          31, 100, 168, 157,  40,  88, 191,  51,  24, 196,  86, 181,  10,
         151, 198, 131, 149, 179, 146,  50,  79, 121, 164, 113,  23, 132,
          12,  74,  93,  69, 192,  35, 174,  25, 108, 148,  34,   2, 165,
         101,  75, 116,  70,   1, 166, 120,  94,  44, 133,  90, 163, 183,
           9, 185,   3,  60,  11,  97,  80, 186,  67, 118,  55,  47, 111,
         155,  38, 159, 154, 178, 126, 195, 160, 117,  72,  36, 167, 184,
         142,  21,  20,  57, 130, 123,  76, 176, 194,  48, 156,   7,  14,
           4, 190,  26, 193,   6, 175,

In [88]:
import itertools

indivs = list(itertools.permutations(range(1, 4)))
val = set()
for indiv in indivs:
    val.add(Task.show_path(np.array(indiv))[1])

[{2: [2, 9]}, {3: [1, 2]}]
[{2: [1, 2, 9]}]
[{2: [2, 9]}, {1: [1, 2]}]
[{3: [3, 9]}, {1: [1, 2, 3]}]
[{2: [2, 9]}, {1: [1, 2]}]
[{2: [2, 9]}, {1: [1, 2]}]


In [89]:
val

{3.0, 4.0, 5.0}

In [87]:
GeneratorPopulaion(5)

[(array([3, 2, 1]), 3.0),
 (array([2, 3, 1]), 4.0),
 (array([2, 3, 1]), 4.0),
 (array([3, 2, 1]), 3.0),
 (array([2, 3, 1]), 4.0)]

In [17]:
test = EncodePriority('D:\MachineLearning\EvolutionComputation\IDPCDU\Datasets\IDPCDU_Edges\set2\idpc_100x200x2296097.idpc')

100%|██████████| 200/200 [04:02<00:00,  1.21s/it]


In [70]:
test = EncodePriority('D:\MachineLearning\EvolutionComputation\IDPCDU\Datasets\IDPCDU_Edges\set2\idpc_100x200x2296097.idpc')

100%|██████████| 200/200 [02:14<00:00,  1.49it/s]


In [71]:
Task = test

In [29]:
GeneratorPopulaion(1)

[(array([127, 198,  38, 180,  45, 123, 168,  70, 166,  10, 163,  90, 184,
          67,   7, 134,  82, 101,  52,  83, 110,  25, 200, 113, 111,  97,
           3,   9, 152, 138,  27,  51, 116, 121,  96,  31,  28,  68, 160,
         146, 119, 142, 195,  93,  63, 179,  87,  34,  69, 154,  33,  44,
          35,  75,  84, 182, 188,  62,   5, 148, 106, 132, 122,  88,  32,
          22, 196, 181,  59,  64,   6, 165,   8, 114, 136, 104, 176, 167,
         183,  56,  91, 186, 139,  73, 108,  76,  53, 137,  49,  46, 150,
          36, 100,  42,  99,  30, 199, 129,  55, 192, 187, 159, 125, 126,
         144,   1,  16, 130, 143, 120,  57, 109,  77, 185, 140, 197, 117,
          40,  24,  54, 171,  20, 193,  98, 169,  41,  78, 175,  17, 151,
         170, 164,  86, 149, 147, 103,  50,   2, 141, 178, 118,  14, 124,
         162, 135,   4,  72,  89,  61,  85,  37, 102, 157,  74, 158,  71,
         107,  12, 133,  92, 172, 161,  47,  58,  66, 112, 194,  43,  94,
         191,  15, 174, 177,  80,  48,

In [31]:
Task.START_NODE

43

In [33]:
num = []
for i in range(1, 201):
    num.append(len(Task.domain_start_nodes[1]))

num.sort()
num

[100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100

In [108]:
import time
start = time.time()
t= Task.Greedy_Cost([127, 198,  38, 180,  45, 123, 168,  70, 166,  10, 163,  90, 184,
          67,   7, 134,  82, 101,  52,  83, 110,  25, 200, 113, 111,  97,
           3,   9, 152, 138,  27,  51, 116, 121,  96,  31,  28,  68, 160,
         146, 119, 142, 195,  93,  63, 179,  87,  34,  69, 154,  33,  44,
          35,  75,  84, 182, 188,  62,   5, 148, 106, 132, 122,  88,  32,
          22, 196, 181,  59,  64,   6, 165,   8, 114, 136, 104, 176, 167,
         183,  56,  91, 186, 139,  73, 108,  76,  53, 137,  49,  46, 150,
          36, 100,  42,  99,  30, 199, 129,  55, 192, 187, 159, 125, 126,
         144,   1,  16, 130, 143, 120,  57, 109,  77, 185, 140, 197, 117,
          40,  24,  54, 171,  20, 193,  98, 169,  41,  78, 175,  17, 151,
         170, 164,  86, 149, 147, 103,  50,   2, 141, 178, 118,  14, 124,
         162, 135,   4,  72,  89,  61,  85,  37, 102, 157,  74, 158,  71,
         107,  12, 133,  92, 172, 161,  47,  58,  66, 112, 194,  43,  94,
         191,  15, 174, 177,  80,  48,  79, 155, 156, 153,  95,  11,  18,
         128, 173,  23,  60,  21, 105,  29,  13,  81, 189, 131,  19,  65,
         145,  26,  39, 190, 115])

end = time.time()

In [110]:
t

115.0

In [107]:
end-start

0.8898766040802002

In [109]:
end-start

0.015924453735351562